# Wprowadzenie do sieci neuronowych i uczenia maszynowego
## Lab: Podstawowe moduły sieci neuronowych w PyTorch

---

**Autorzy materiałów:** Piotr Baryczkowski, Jakub Bednarek<br>

---

## Uwaga

* **Aby wykonać polecenia należy najpierw przejść do trybu 'playground'. File -> Open in Playground Mode**
* Nowe funkcje Colab pozwalają na autouzupełnianie oraz czytanie dokumentacji.


## Cel ćwiczeń:

* zapoznanie się z pojęciem **zbioru danych** i jego charakterystyką,
* wykorzystanie podstawowych warstw neuronowych,
* implementacja procesu uczenia sieci neuronowej + *good practices*

In [ ]:
import numpy as np
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Zbiór danych

### Wprowadzenie oraz popularne zbiory danych

Odpowiednie przygotowanie zbioru danych odgrywa znaczącą rolę w uczeniu sieci neuronowych. Zazwyczaj zbiory danych zawierają 3 pozbiory:

* treningowy - wykorzystywany do uaktualniania wag modelu neuronowego,
* walidacyjny - do oceny modelu po każdej **epoce**,
* testowy - do porównania modelu z innymi rozwiązaniami.

**Uwaga:** bardzo często popularne zbiory danych nie posiadają zbioru testowego, ponieważ nie prowadzą tzw. **leaderboard**.

Najpopularniejsze zbiory danych:

* **MNIST**,
* eMNIST,
* Caltech 101/256,
* Cityscapes,
* Kitty,
* LFW Face Dataset,
* ImageNet

Więcej informacji: [wiki](https://en.wikipedia.org/wiki/List_of_datasets_for_machine-learning_research) [kaggle](https://www.kaggle.com/datasets) [google](https://toolbox.google.com/datasetsearch)

### Obsługa zbioru danych w PyTorch

Kod do przetwarzania danych może stać się nieczytelny i trudny do utrzymania; idealnie chcielibyśmy, aby kod związany z naszym zestawem danych był oddzielony od kodu, który odpowiada za uczenie modelu. PyTorch udostępnia dwie klasy do obsługi danych: `torch.utils.data.DataLoader` i `torch.utils.data.Dataset`, które pozwalają na użycie zarówno gotowych zestawów danych, jak i własnych. `Dataset` przechowuje próbki i ich odpowiadające etykiety, a `DataLoader` jest swego rodzaju nakładką na obiekt `Dataset`, umożliwiając łatwy dostęp do próbek - ładowanie danych, dzielenie danych na podzbiory (_ang. batch_). [link](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

#### Tworzenie zbioru danych

In [ ]:
dataset_size = 10

# zbiór danych składający się z losowych obrazków o rozmiarze (32, 32, 3) oraz etykiet po kolei od 0 do dataset_size
x = [np.random.uniform(size=(32, 32, 3)) for _ in range(dataset_size)]
y = [i for i in range(dataset_size)]

tensor_x = torch.Tensor(x)
tensor_y = torch.Tensor(y)

# utworzenie "iteratora" zbioru danych
dataset = torch.utils.data.TensorDataset(tensor_x, tensor_y)
dataloader = torch.utils.data.DataLoader(dataset)

#### Iterowanie po zbiorze danych

In [ ]:
for x, y in dataloader:
  print(x.shape, y)

#### Tasowanie

In [ ]:
dataloader_shuffled = torch.utils.data.DataLoader(dataset, shuffle=True)

for x, y in dataloader_shuffled:
  print(x.shape, y)

#### Mapowanie

In [ ]:
class MappedDataset(torch.utils.data.Dataset):
  def __init__(self, dataset, transform_func):
    self.dataset = dataset
    self.transform_func = transform_func

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    data, label = self.dataset[idx]
    return self.transform_func(data, label)


def map(x, y):
  y = y * 2
  return x, y


dataset_mapped = MappedDataset(dataset, map)
dataloader_mapped = torch.utils.data.DataLoader(dataset_mapped)

for x, y in dataloader_mapped:
  print(x.shape, y)

#### Filtrowanie

In [ ]:
class FilteredDataset(torch.utils.data.Dataset):
  def __init__(self, dataset, threshold):
    self.filtered_data = [
      (data, label) for data, label in dataset if label > threshold
    ]

  def __len__(self):
    return len(self.filtered_data)

  def __getitem__(self, idx):
    return self.filtered_data[idx]


dataset_filtered = FilteredDataset(dataset, 5)

dataloader_filtered = torch.utils.data.DataLoader(dataset_filtered)

for x, y in dataloader_filtered:
  print(x.shape, y)

#### Grupowanie

In [ ]:
dataloader_batch = torch.utils.data.DataLoader(dataset, batch_size=5)

for x, y in dataloader_batch:
  print(x.shape, y)

#### Składanie wielu operacji na raz

In [ ]:
dataset_mix = MappedDataset(dataset, map)
dataset_mix = FilteredDataset(dataset_mix, 6)

dataloader_mix = torch.utils.data.DataLoader(dataset_mix, shuffle=True, batch_size=5)

for x, y in dataloader_mix:
  print(x[0, 0, 0, 0], x.shape, y)

#### Zadanie 1

Stwórz zbiór danych (bez podziału na zbiory treningowe, walidacyjne i testowy) składający się z 10000 elementów, zawierający pary (x, y) danych
dla funkcji **sinus**. Dane x niech będą z zakresu [-2 * PI, 2 * PI], y - odpowiadające im wartości funkcji sinus.

Następnie utwórz providera za pomocą PyTorch Dataset API, który będzie:

* tasował dane
* mapował tak, aby dane x, z zakresu [-2 \* PI, 0), były transformowane do przedziału [0, 2 \* PI)

Podpowiedź: (x + 2PI) % 2PI,
* grupował dane w batche o rozmiarze 32

In [ ]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader


dataset_size = 10000

x = ...
y = ...

def transform_sinus(x, y):
  return ...

dataset = ...
mappedDataset = ...
dataloader = ...

for x, y in dataloader:
  print(x, y)

### Popularne zbiory danych

Biblioteka PyTorch zawiera gotowe funkcje wczytujące dla niektórych zbiorów danych. Większość gotowych zbiorów danych możemy znaleźć w bibliotece *torchvision*. Tutaj możesz znaleźć dokładną listę dostępnych zbiorów danych - [link](https://pytorch.org/vision/stable/datasets.html)

Jednym z popularniejszych zbiorów danych jest MNIST. Jest to zbiór zawierający cyfry pochodzące z pisma odręcznego wraz z ich przypisanymi etykietami ('1', '2', etc.). Poniżej przykładowe pobranie i wykorzystanie zbioru MNIST.

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

## Podstawowe warstwy neuronowe

Przy projektowaniu sieci neuronowych możemy wyróżnić podstawowe operacje (warstwy), które się powtarzają. Dla wygody, PyTorch zawiera gotowe implementacje najprostszych z nich, oraz udostępnia interfejsy do tworzenia własnych, bardziej skomplikowanych.

Operacje możemy podzielić na:
* **uczalne** - zawierające zmienne uczalne (np. *w* i *b* w warstwie w pełni połączonej),
* **nieuczalne** - takie, które wykonują pewne charakterystyczne działania na danych, jednak nie potrzebują do tego zmiennych, które będą uczone w trakcie propagacji gradientu.

Poniżej zaprezentowane zostały popularne operacje uczalne i nieuczalne.

**Uwaga**
Wszystkie operacje są reprezentowane jako klasy.

### Uczalne



Dzięki predefiniowanym warstwom nie ma potrzeby samodzielnej deklaracji nowych zmiennych. Wszystkie zmienne uczone są deklarowane (zgodnie z implementacją danej warstwy) wewnątrz obiektu, a następnie przechowywane.

Do zmiennych uczonych można dostać się poprzez własność *state_dict* (lub *parameters*).

In [ ]:
m = torch.nn.Linear(5, 5)

m.state_dict()

#### Linear

Jest to warstwa w pełni połączona, która pobiera jako wejście wektor i produkuje na wyjściu wektor o długości równej rozmiarowi warstwy (liczby neuronów).

In [ ]:
# definicja warstwy
linear1 = torch.nn.Linear(8, 2)

# inferencja
x = torch.ones([8])
y = linear1(x)

# rozmiar wejściowego oraz wyjściowego tensora
print(x.shape, y.shape)

# zmienne uczone
state_dict = linear1.state_dict()
print(f"Wagi w warstwie linear: {state_dict['weight']}")
print(f"Bias'y w warstwie linear: {state_dict['bias']}")

#### Convolution

Warstwa w pełni połączona dobrze sprawdza się przy danych jednowymiarowych. W przypadku danych wielowymiarowych (jak obrazy) korzystanie z nich byłoby bardzo kosztowne obliczeniowo. Aby wykonać pojedynczą operację *Linear* z 128 neuronami na obrazie o rozmiarach (256, 256, 3) należałoby zadeklarować  256 * 256 * 3 * 128 = 25165824 zmiennych uczonych.

Popularnym rozwiązaniem efektywnego przetwarzania danych wielowymiarowych są operacje konwolucji ([link do wizualizacji](https://github.com/vdumoulin/conv_arithmetic)). Konwolucja (inaczej splot) w sieciach neuronowych intuicyjnie jest, tak samo jak *Linear*, kombinacją liniową danego podobszaru danych wielowymiarowych i zmiennych uczonych (inaczej *kernel*).

Konwolucja w PyTorch posiada szereg parametrów takich jak:
* liczba kanałów wejściowych - liczba kanałów w obrazie wejściowym,
* liczba kanałów wyjściowych - liczba kanałów "wyprodukowana" przez konwolucję,
* kernel_size - rozmiar kernela,
* stride - "rozstrzał" przetwarzanego podobszaru (patrz link do github),
* padding - dopełnienie dodane do wszystkich czterech stron danych wejściowych. Domyślnie: 0,

W porównaniu do przykładu przytoczonego powyżej, konwolucja z 128 filtrami, rozmiarem kernela równym (3, 3), dla takich samych danych wejściowych zawierałaby 3 * 3 * 3 * 128 = 3456, czyli ponad 7281 (!) razy mniej niż w przypadku *Linear*. Ponadto, w przetwarzaniu danych, w których zachodzą lokalne zależności (na obrazie sąsiadujące piksele reprezentują zazwyczaj ten sam obiekt) konwolucja sprawdza się o wiele lepiej niż *Linear*.

In [ ]:
# definicja warstwy
conv1 = torch.nn.Conv2d(128, 128, (3, 3), (2, 2), 1)

# inferencja (przy pierwszym wywołaniu warstwy Linear1 zostaną stworzone zmienne uczone)
x = torch.ones([10, 128, 128, 3])
y = conv1(x)

# rozmiar wejściowego oraz wyjściowego tensora
print(x.shape, y.shape)

# zmienne uczone
state_dict = conv1.state_dict()
print(f"Wagi w warstwie conv: {state_dict['weight'].shape}")
print(f"Bias'y w warstwie conv: {state_dict['bias'].shape}")

### Nieuczalne

Warstwy nieuczalne zazwyczaj wykonują pewne operacje techniczne, typu zmiana kształtu, skalowanie danych, lub są wykorzystywane w **regularyzacji** (o czym będzie mowa na kolejnych zajęciach).

#### Pooling

Jest to jedna z popularniejszych metod regularyzacji, polegająca na wykonaniu pewnej operacji na małym wycinku danych. Przykładowo MaxPooling2D, podobnie jak konwolucja 2D (patrz wizualizacje), wybiera podobszar obrazu o jakichs wymiarach (np. 2x2) a następnie wybiera maksymalny obiekt z tego okna, tworząc nowy obraz (np. zmniejszony 2-krotnie). Istnieją również inne metody poolingu:

* average - z okna obliczana jest średnia,
* median - z okna obliczana jest mediana,
* minimum - z okna wybierana jest najmniejsza wartość,
* itp.

In [ ]:
# definicja warstwy
mp1 = torch.nn.MaxPool2d((2, 2), (2, 2))

# inferencja (przy pierwszym wywołaniu warstwy Linear1 zostaną stworzone zmienne uczone)
x = torch.ones([10, 128, 128, 3])
y = mp1(x)

# rozmiar wejściowego oraz wyjściowego tensora
print(x.shape, y.shape)

#### Flatten

Flatten jest prostą funkcją spłaszczającą **każdy element w batchu**. Przykładowo dla grupy 10 obrazów o pewnych wymiarach wyprodukowanych zostanie 10 wektorów (spłaszczonych do wektorów obrazów).

In [ ]:
# definicja warstwy
ft1 = torch.nn.Flatten()

# inferencja (przy pierwszym wywołaniu warstwy Linear1 zostaną stworzone zmienne uczone)
x = torch.ones([10, 128, 128, 3])
y = ft1(x)

# rozmiar wejściowego oraz wyjściowego tensora
print(x.shape, y.shape)

## Proces uczenia sieci neuronowej

Proces uczenia sieci neuronowych składa się z kilku części. Po inicjalizacji modelu oraz zbioru danych następuje uczenie modelu składające się z wielu **epok**. Epoka to pojedyncze przeiterowanie po całym zbiorze danych (podzbiory treningowe i walidacyjne). Przy czym model jest uczony (bład jest propagowany) tylko na zbiorze treningowym. **Nigdy na zbiorach walidacyjnym i testowym.** Proces uczenia sieci neuronowej składa się (najczęściej) następujących części:

1. Inicjalizacja modelu,
2. Inicjalizacja zbioru danych,
3. Pętla treningowa,
  1. Uczenie na zbiorze treningowym (raz!),
  2. Ocena modelu na zbiorze walidacyjnym,
4. Ocena modelu na zbiorze testowym (opcjonalne)

#### Inicjalizacja modelu

**Uwaga:** Jeśli model składa się z następujących po sobie operacji, można opakować go dla wygody w strukturę *Sequential*, tak jak pokazano poniżej.

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
import torch.nn as nn
from torch.nn.modules.activation import ReLU


class MyModel(nn.Module):
    def __init__(self) -> None:
        """Model który na wejściu otrzymuje obraz a na wyjściu produkuje skalar

        W skład modelu wchodzą 3 warstwy konwolucyjne o rozmiarach 64, 32, 16 (out_channels),
        każda z rozmiarem kernela 5x5 oraz stride 2x2 (czyli obraz po każdej warstwie będzie 2 razy mniejszy)
        potem następuje spłaszczenie obrazu do wektora i przetwarzanie warstwami w pełni połączonymi.
        Wszystkie warstwy (oprócz wyjściowej) korzystają z funkcji aktywacji 'relu'
        """
        super(MyModel, self).__init__()
        self.conv_relu_stack = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=5, stride=2, padding=1),
            ReLU(),
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=5, stride=2, padding=1),
            ReLU(),
            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=5, stride=2, padding=1),
            ReLU(),
        )
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=64, out_features=32),
            ReLU(),
            nn.Linear(in_features=32, out_features=10),
            ReLU(),
            nn.Linear(in_features=10, out_features=10),
        )

    def forward(self, x: torch.Tensor):
        x = self.conv_relu_stack(x)
        x = self.flatten(x)
        return self.linear_relu_stack(x)


model = MyModel()
print(model)

#### Inicjalizacja zbioru danych

Jako zbiór danych wykorzystany zostanie zaprezentowany wcześniej zbiór **MNIST**.

In [ ]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

### Proces uczenia sieci neuronowej

In [ ]:
learning_rate = 1e-3
batch_size = 32
epochs = 5

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

#### Zadanie 2

Stwórz sieć neuronową składającą się z:

* 3 warstw konwolucyjnych (kernel size = 5, liczba filtrów = 128, 64, 32, **bez stride (stride=1)**, aktywacja = relu, **padding='1'**)
* 3 warstw Max Pooling-u, każda znajdująca się za kolejną warstwą konwolucyjną (pool size = 2, **stride = 2**, padding=1, funkcja aktywacji: relu) (tzn. Conv2D -> MaxPooling2D -> Conv2D -> MaxPooling2D -> ...),
* 3 warstw w pełni połączonych (rozmiary: 512, 128, **liczba klas**, funkcja aktywacji: relu)

Przetestuj swoją sieć na następujących zbiorach danych:

* Cifar 10
* Cifar 100

In [ ]:
import numpy as np
import torch
import torch.nn as nn

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int) -> None:
        super(NeuralNetwork, self).__init__()
        self.blocks = nn.Sequential(
            # TODO
        )

    def forward(self, x: torch.Tensor):
        return self.blocks(x)


model = NeuralNetwork(10)
print(model)


In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

# TODO - init dataset

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32

# TODO - init dataloaders

In [ ]:
# TODO - train networks

## Pretrenowane sieci

Biblioteka udostępnia również **pretrenowane** modele. Tzn. takie, które zostały już wyuczone na pewnych zbiorach danych.

Istnieje wiele znanych, pretrenowanych sieci neuronowych:

* ResNet (w wersji 50, 100, itp.),
* Inception (V2, V3),
* VGG (16, 19),
* MobileNet,
* LinearNet

Poniżej zaprezentowany został przykład użycia jednej z nich.

**Uwaga:** wgrywanie plikow działa poprawnie w przeglądarce google chrome. Na przeglądarkach FireFox można bezpośrednio wgrać pliki korzystając z zakładki "files" po lewej strone (rozwijany pasek nawigacji - strzałka w prawo). Taki plik można wczytać bezpośrednio korzystając z: Image.open('path_to_file.png').

**Uwaga:** czasem niezbędne jest dwukrotne wykonanie poniższego skryptu, aby można było skorzystać z widgetu do wgrywania pliku.

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

# Using pretrained weights:
resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
resnet50(weights="IMAGENET1K_V1")
resnet50(pretrained=True)  # deprecated
resnet50(True)  # deprecated

# Using no weights:
resnet50(weights=None)
resnet50()
resnet50(pretrained=False)  # deprecated
resnet50(False)  # deprecated

In [ ]:
import numpy as np
from google.colab import files
from PIL import Image
from io import BytesIO
import IPython.display

# wybranie sciezki do pliku (jakikolwiek obraz z google)
uploaded = files.upload()

# wyświetlenie obrazu
im = Image.open(BytesIO(uploaded['cat.jpg']))
# dla przeglądarek FireFox
# im = Image.open('cat.jpg')
IPython.display.display(im)

In [ ]:
# przygotowanie obrazka
im_numpy = np.array(im.resize((224, 224)))  # rozmiar który przyjmuje ResNet
im_torch = torch.Tensor(im_numpy).permute(2, 0, 1).unsqueeze(0)

# załadowanie pretrenowanego modelu i jego inferencja
from torchvision.models import resnet50, ResNet50_Weights
weights = ResNet50_Weights.IMAGENET1K_V1
model = resnet50(weights=weights)
model.eval()

print(im_torch.shape)
pred = model(im_torch)

# pobranie wyniku z sieci neuronowej
print(torch.argmax(pred, -1).numpy())

Zgodnie z klasami zawartymi w zbiorze danych ImageNet, odpowiedź 284 oznacza "Kot syjamski".

Przykładowe klasy:

* 21: 'kite',
* 22: 'bald eagle, American eagle, Haliaeetus leucocephalus',
* 23: 'vulture',
* 243: 'bull mastiff',
* 244: 'Tibetan mastiff',
* 245: 'French bulldog',
* 282: 'tiger cat',
* 283: 'Persian cat',
* 284: 'Siamese cat, Siamese',
* 285: 'Egyptian cat',

Wszystkie obecne w ImageNet klasy: [link](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a)

#### Zadanie 3

Spróbuj wykorzystać dowolną inną pretrenowaną sieć neuronową, zgodnie z powyższym schematem. Możesz spróbować załadować inny obraz.

Dostępne pretrenowane modele: [link](https://pytorch.org/vision/stable/models.html)

**Uwaga:** zwróć uwagę na rozmiary danych, które przyjmują poszczególne sieci neuronowe.